# Advent of Code
Charel Felten

In [2]:
import pandas as pd
import numpy as np

## 1. Advent

In [3]:
df = pd.read_csv("inputs/1", header=None)

f = lambda w: (df.rolling(w).sum().dropna().diff().fillna(0) > 0).sum().loc[0]

print(f"Task 1: {f(1)}, Task 2: {f(3)}")

Task 1: 1266, Task 2: 1217


## 2. Advent

In [4]:
df = pd.read_csv("inputs/2", header=None, sep=" ", names=["direction", "distance"])

t1 = df.groupby("direction").sum().apply(lambda col: (col["down"] - col["up"]) * col["forward"]).loc["distance"]

h, d, a = 0,0,0
for _, (dir, dis) in df.iterrows():
    if dir == "down": a += dis
    if dir == "up": a -= dis
    if dir == "forward":
        h += dis
        d += a*dis
        
print(f"Task 1: {t1}, Task 2: {h * d}")

Task 1: 1882980, Task 2: 1971232560


## 3. Advent

In [8]:
df = pd.read_csv("inputs/3", header=None, sep=" ", dtype="str")

t1 = df\
    .apply(lambda r: list(r.iloc[0]), result_type="expand", axis=1)\
    .agg(["median", lambda c: int(not(c.median()))])\
    .apply(lambda c: 2**((len(df.iloc[0,0])-1)-c.name) * c)\
    .sum(axis=1).prod().astype(int)

t2 = df.apply(lambda r: list(r.iloc[0]), result_type="expand", axis=1).astype(int)

def x(t2, b):
    for c in t2: t2 = t2[t2[c] == t2.groupby(c).size().sort_index(ascending=b).sort_values(ascending=b).index[0]]
    return t2.apply(lambda c: 2**((len(df.iloc[0,0])-1)-c.name) * c).sum(axis=1).iloc[0]

t2 = x(t2, False) * x(t2, True)

print(f"Task 1: {t1}, Task 2: {t2}")

Task 1: 4147524, Task 2: 3570354


## 4. Advent

In [135]:
inputs = pd.read_csv("inputs/4t", header=None, nrows=1).to_numpy().reshape(-1)

boards = pd.read_csv("inputs/4t", header=None, skiprows=[0], sep=r" +").to_numpy().reshape(-1, 5, 5)

bingos = np.zeros_like(boards, dtype=bool)

for inp in inputs:
    for match in np.argwhere(boards == inp): bingos[tuple(match)] = True
    hs = np.argwhere(bingos.sum(axis=1) == 5)
    vs = np.argwhere(bingos.sum(axis=2) == 5)

    if np.any(vs):
        w = vs[0,0]
        break
    elif np.any(hs):
        w = hs[0,0]
        break

t1 = inp * boards[w, ~bingos[w]].sum()

print(t1)

4512
